In [ ]:
from prefect import task, Flow
from prefect.triggers import manual_only, any_failed
from prefect.tasks.control_flow import ifelse


@task
def pull_critical_data():
    "Pulls and cleans data, which is then returned."
    return 1


@task
def build_report(data):
    "Compiles data into a report which is saved somewhere."
    return 'quality report'


@task(trigger=manual_only)
def email_report_to_board(report):
    """
    Emails `report` as an attachment to entire board of directors.
    
    Returns `True` if email is sent correctly, `False` otherwise.
    """
    return True


@task(trigger=any_failed)
def complain_to_data_analyst():
    "Alerts the appropriate person to review reporting code."
    pass


with Flow() as f:
    data = pull_critical_data()
    report = build_report(data)
    board_email = email_report_to_board(report)
    complaint = complain_to_data_analyst(upstream_tasks=[board_email])


In [ ]:
f.set_reference_tasks([board_email])


In [ ]:
flow_state = f.run(return_tasks=[report, board_email, complaint])

print("Flow state: {}\n".format(flow_state))
print(flow_state.result)

##    Flow state: Pending("Some terminal tasks are still pending.")
    
##    Flow results: {
##     <Task: build_report>: Success("Task run succeeded."), 
##     <Task: email_report_to_board>: Pending("Trigger function is "manual_only""), 
##     <Task: complain_to_data_analyst>: Pending()
##     }


In [ ]:
flow_state.result[report].result
### 'quality report'


In [ ]:
new_flow_state = f.run(task_states=flow_state.result, 
                       start_tasks=[board_email], 
                       return_tasks=[board_email, complaint])

print("Flow state: {}\n".format(new_flow_state))
print(new_flow_state.result)

##    Flow state: Success("All reference tasks succeeded.")
    
##    Flow result: { 
##       <Task: email_report_to_board>: Success("Task run succeeded."), 
##       <Task: complain_to_data_analyst>: TriggerFailed("Trigger was "any_failed" but none of the upstream tasks failed.")
##       }
